In [132]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Conv1D, GlobalMaxPooling1D, Flatten, concatenate, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD

import torch
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import numpy as np
import pickle

import pandas as pd

In [133]:
with open("model_raw_data_2", "rb") as fp:
    model_raw_data = pickle.load(fp)
X_train, X_train_bart, X_test, X_test_bart, y1_train, y1_test, y2_train, y2_test = model_raw_data

In [134]:
with open("val_raw_data_2", "rb") as fp:
    val_raw_data = pickle.load(fp)
X_trail, X_trail_bart, y1_trail, y2_trail = val_raw_data

In [135]:
def proccess_bart_data(clause_vectors_res):
    tf.squeeze(clause_vectors_res[0], axis=0)
    clause_vectors_res_2d = [tensor.squeeze(0) for tensor in clause_vectors_res]
    clause_vectors_padded = pad_sequence(clause_vectors_res_2d, batch_first=True)
    return clause_vectors_padded

In [136]:
# def proccess_y_data(output):
#     encoder = OneHotEncoder(sparse_output=False)
#     output_reshaped = np.array(output).reshape(-1, 1)
#     output_one_hot = encoder.fit_transform(output_reshaped)
#     return output_one_hot
def process_y_data(output):
    encoder = OneHotEncoder(sparse_output=False)
    output_reshaped = np.array(output).reshape(-1, 1)
    output_one_hot = encoder.fit_transform(output_reshaped)
    return output_one_hot, encoder

In [137]:
X = X_train_bart + X_test_bart + X_trail_bart
X = proccess_bart_data(X)
X_train_bart_p = X[:len(X_train_bart)]
X_test_bart_p = X[len(X_train_bart):len(X_train_bart)+len(X_test_bart)]
X_trail_bart_p = X[len(X_train_bart)+len(X_test_bart):]

# without # after category
y1_train = [category.split('#')[0] for category in y1_train]
y1_test = [category.split('#')[0] for category in y1_test]
y1_trail = [category.split('#')[0] for category in y1_trail]

y1_train_hot, encoder1 = process_y_data(y1_train)
y2_train_hot, encoder2 = process_y_data(y2_train)


y1_test_hot = encoder1.transform(np.array(y1_test).reshape(-1, 1))
y2_test_hot = encoder2.transform(np.array(y2_test).reshape(-1, 1))
y1_trail_hot = encoder1.transform(np.array(y1_trail).reshape(-1, 1))
y2_trail_hot = encoder2.transform(np.array(y2_trail).reshape(-1, 1))

# y1_train_hot = proccess_y_data(y1_train)
# y1_test_hot = proccess_y_data(y1_test)
# y1_trail_hot = proccess_y_data(y1_trail)

# y2_train_hot = proccess_y_data(y2_train)
# y2_test_hot = proccess_y_data(y2_test)
# y2_trail_hot = proccess_y_data(y2_trail)

proccess_data = (X_train_bart_p, X_test_bart_p, y1_train_hot, y1_test_hot, y2_train_hot, y2_test_hot, encoder1, encoder2)
with open("model_proccess_data_3", "wb") as fp:   #Pickling
    pickle.dump(proccess_data, fp)

test_output_data = (X_train, X_test, y1_train, y1_test, y2_train, y2_test)
with open("test_output_data_3", "wb") as fp:   #Pickling
    pickle.dump(test_output_data, fp)


val_output_data = (X_trail, X_trail_bart_p, y1_trail_hot, y2_trail_hot, y1_trail, y2_trail)
with open("val_output_data_3", "wb") as fp:   #Pickling
    pickle.dump(val_output_data, fp)